In [2]:
!pip install pyautogen


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
openAI_API_Key=os.environ['openAI_API_Key']
llm_config = {"model": "gpt-3.5-turbo"}

In [4]:
import autogen

C:\Users\moham\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
C:\Users\moham\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
from autogen import ConversableAgent

In [6]:
agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

Note:  human_input_mode="NEVER" means the agent doesnot need human response before go ahead and it continues by it self!

In [7]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure, here's a joke for you:

Why don't scientists trust atoms?

Because they make up everything!


In [8]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Sure, please provide me with the joke you'd like me to repeat.


As is clear we do not have any conversation. How can we make a conversation among the agents?

**Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained**

In [9]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [10]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, great to meet you! Let's dive into some laughs together. So, I recently learned that laughter is the best medicine. Unless you're laughing for no reason, then they just call it weird.

--------------------------------------------------------------------------------
joe (to cathy):

Well, Cathy, I agree with you on that! It's weird unless you're a stand-up comedian, then they call it a job.

--------------------------------------------------------------------------------
cathy (to joe):

Haha, you got that right, Joe! Being a stand-up comedian means I get to make people laugh for a living. So, in a way, my job description is basically "professional weirdo."

--------------------------------------------------------------------------------


**Printing Results**

You can print out:

Chat history

Cost

Summary of the conversation

In [11]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': "Hey Joe, great to meet you! Let's dive into some laughs "
             'together. So, I recently learned that laughter is the best '
             "medicine. Unless you're laughing for no reason, then they just "
             'call it weird.',
  'role': 'user'},
 {'content': "Well, Cathy, I agree with you on that! It's weird unless you're "
             'a stand-up comedian, then they call it a job.',
  'role': 'assistant'},
 {'content': 'Haha, you got that right, Joe! Being a stand-up comedian means I '
             'get to make people laugh for a living. So, in a way, my job '
             'description is basically "professional weirdo."',
  'role': 'user'}]


In [12]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 114,
                                                             'cost': 0.000295,
                                                             'prompt_tokens': 248,
                                                             'total_tokens': 362},
                                      'total_cost': 0.000295},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 114,
                                                             'cost': 0.000295,
                                                             'prompt_tokens': 248,
                                                             'total_tokens': 362},
                                      'total_cost': 0.000295}}


In [13]:
pprint.pprint(chat_result.summary)

('Haha, you got that right, Joe! Being a stand-up comedian means I get to make '
 'people laugh for a living. So, in a way, my job description is basically '
 '"professional weirdo."')


**Get a better summary of the conversation**

In [14]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, great to meet you! Let's dive into some laughs together. So, I recently learned that laughter is the best medicine. Unless you're laughing for no reason, then they just call it weird.

--------------------------------------------------------------------------------
joe (to cathy):

Well, Cathy, I agree with you on that! It's weird unless you're a stand-up comedian, then they call it a job.

--------------------------------------------------------------------------------
cathy (to joe):

Haha, you got that right, Joe! Being a stand-up comedian means I get to make people laugh for a living. So, in a way, my job description is basically "professional weirdo."

--------------------------------------------------------------------------------


In [15]:
pprint.pprint(chat_result.summary)

("Laughter is the best medicine, unless you're laughing for no reason, then "
 "it's just weird or part of being a stand-up comedian.")


**Chat Termination**

Chat can be terminated using a termination conditions.

In [16]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [17]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Why was the math book sad? Because it had too many problems!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! I love math jokes. So, have you heard about the claustrophobic astronaut? He just needed a little space!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that's a great one, Joe! I guess he needed some breathing room up there!

--------------------------------------------------------------------------------
joe (to cathy):

Exactly, Cathy! He needed to space out a bit more. So, tell me, do you have any good jokes up your sleeve?

--------------------------------------------------------------------------------
cathy (to joe):

Of course, Joe! How does a penguin build its hous

In [18]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was: How does a penguin build its house? It igloos it together!

--------------------------------------------------------------------------------
cathy (to joe):

Thanks, Joe! Have a fantastic day, and keep those jokes coming!

--------------------------------------------------------------------------------
joe (to cathy):

Thank you, Cathy! I'll keep the jokes coming for sure. Have a fantastic day too! Take care!

--------------------------------------------------------------------------------
cathy (to joe):

You too, Joe! Bye now!

--------------------------------------------------------------------------------
joe (to cathy):

Goodbye, Cathy! See you soon!

--------------------------------------------------------------------------------
cathy (to joe):

I gotta go.

---------------------------------------